# Dataset Data Loader

#Previously was Batch gradient descent


  #Memory inefficient load everything everytime

  
  #Convergence poor

# Divide into multiple batches now 
          
          #It is called mini batch Gradient Descent

#Try on synthetic data

In [10]:
from sklearn.datasets import make_classification
import torch
# Step 1: Create a synthetic classification dataset using sklearn
x, y = make_classification(
    n_samples=10,       # Number of samples
    n_features=2,       # Number of features
    n_informative=2,    # Number of informative features
    n_redundant=0,      # Number of redundant features
    n_classes=2,        # Number of classes
    random_state=42 
    )    # For reproducibility

In [11]:
# Convert the data to PyTorch tensors
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Dataset class

In [12]:
from torch.utils.data import Dataset, DataLoader

In [13]:
class CustomDataset(Dataset): 
    #inherit from Dataset class

    def __init__(self,features,labels):
        self.features = features
        self.labels =labels

    def __len__(self):
        
        return self.features.shape[0]
    
    def __getitem__(self,index):

        return self.features[index], self.labels[index]
    
    

In [14]:
dataset = CustomDataset(x,y)

In [15]:
len(dataset)

10

In [16]:
dataset[0]

(tensor([ 1.0683, -0.9701]), tensor(1))

# Data loader

In [ ]:
#Dataloader object
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [24]:
for batch_features, batch_labels in dataloader:
    print(batch_features)
    print(batch_labels)
    print("-"*50)

tensor([[-2.8954,  1.9769],
        [ 1.7774,  1.5116]])
tensor([0, 1])
--------------------------------------------------
tensor([[-0.9382, -0.5430],
        [-0.7206, -0.9606]])
tensor([1, 0])
--------------------------------------------------
tensor([[-0.5872, -1.9717],
        [ 1.8997,  0.8344]])
tensor([0, 1])
--------------------------------------------------
tensor([[-1.9629, -0.9923],
        [-1.1402, -0.8388]])
tensor([0, 0])
--------------------------------------------------
tensor([[ 1.0683, -0.9701],
        [ 1.7273, -1.1858]])
tensor([1, 1])
--------------------------------------------------


#Try on real data

In [ ]:
#Data preprocessing
import numpy as np
import pandas as pd
import torch
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
# deleting the 'id' column  and Unnamed: 32 column
df.drop(columns=['id', 'Unnamed: 32'], inplace =True)

x = df.iloc[:, 1:]
y = df.iloc[:, 0]

from sklearn.model_selection import train_test_split
x_train, x_test ,y_train, y_test = train_test_split(x,y, test_size=0.2)

from sklearn.preprocessing import StandardScaler
scaler =StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

import torch
x_train_tensor  = torch.from_numpy(x_train).float()
x_test_tensor   = torch.from_numpy(x_test).float()
y_train_tensor  = torch.from_numpy(y_train).float()
y_test_tensor   = torch.from_numpy(y_test).float()

#Dataset and Dataloader

# Model

In [ ]:
import torch
import torch.nn as nn

class MySimpleNN(nn.Module):
    
    def __init__(self, num_features):

        super().__init__()

        self.network = nn.Sequential(nn.Linear
            (num_features,3),
             nn.ReLU(),
             nn.Linear(3,2),
             nn.ReLU(),
             nn.Linear(2,1),
             nn.Sigmoid()
        )
    
    #Forward pass
    def forward(self, features):

        out = self.network(features)
      

        return out
    


In [ ]:
batch_size = 32
learning_rate =0.1
epochs =25
loss_function = nn.BCELoss()

In [ ]:
#Create model
model = MySimpleNN(x_train_tensor.shape[1])

#Define optimizer

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)


#define loop
for epoch in range(epochs):

    #Forward pass
    y_pred = model(x_train_tensor)

    #Loss calculation
    loss = loss_function(y_pred, y_train_tensor.reshape(-1,1)) # or .view(-1,1)

     #clear  gradients
    optimizer.zero_grad()

    
    # Backward Pass
    loss.backward()

    #Parameters Update ( weights and bias)
    optimizer.step()


   

    #Print loss in each epochs
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}') 


In [ ]:
#Model evaluation
with torch.no_grad():
    y_pred = model.forward(x_test_tensor)
    y_pred = (y_pred >= 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f'Accuracy: {accuracy.item()}')

In [ ]:
from torchinfo import summary

summary(
    model,
    input_size=(1, x_train_tensor.shape[1])  # (batch_size, num_features)
)
